---
title: Numerical Analysis (optional, not for exam)
math:
    '\abs': '\left\lvert #1 \right\rvert'
    '\orm': '\left\lvert #1 \right\rvert'
    '\Set': '\left\{ #1 \right\}'
    '\set': '\operatorname{set}'   
    '\mc': '\mathcal{#1}'
    '\M': '\boldsymbol{#1}'
    '\R': '\mathsf{#1}'
    '\RM': '\boldsymbol{\mathsf{#1}}'
    '\op': '\operatorname{#1}'
    '\E': '\op{E}'
    '\d': '\mathrm{\mathstrut d}'
    '\SFM': '\operatorname{SFM}'
    '\utag': '\stackrel{\text{(#1)}}{#2}'
    '\uref': '\text{(#1)}'
    '\minimal': '\operatorname{minimal}'
abstract: |
    An important engineering application of computer science is numerical analysis. This lecture introduces estimation using Monte Carlo simulations and basic linear algebra for solving multiple linear equations. Readers will learn to use NumPy arrays instead of lists to implement numerical analysis efficiently, incorporating the concepts of universal functions and broadcasting.
---

In [2]:
import math
import random
import numpy as np
import ipywidgets as widgets

%matplotlib widget
%reload_ext divewidgets

## Monte Carlo simulation

**What is Monte Carlo simulation?**

- A Monte Carlo simulation is a model used to predict the probability of different outcomes when the intervention of random variables is present.
- Monte Carlo simulations help to explain the impact of risk and uncertainty in prediction and forecasting models.
- A variety of fields utilize Monte Carlo simulations, including finance, engineering, supply chain, and science.

As a simple illustration, we will compute the value of $\pi$:

In [3]:
math.pi

3.141592653589793

Instead of using `pi` from `math`, can we compute $\pi$ directly? Consider a seemingly unrelated math problem:

::::{admonition} What is the chance a random point in a square lies in the inscribed circle?
:class: dropdown

Suppose the square has length $\ell$. The chance is

$$ P[\text{Random point in inscribed circle}] = \frac{\text{area of circle}}{\text{area of square}} = \frac{\pi (\ell /2)^2}{ (\ell)^2 } = \frac{\pi}4 $$

independent of the length $\ell$.

::::

::::{figure} images/RandomPointInSquare.gif
:name:
:alt: Random points in square.
:align: left

Uniformly random points in a square. Green (Red) points are inside (outside) the inscribed circle. ([source code](https://dive4dec.github.io/optlite/#code=from%20manim%20import%20*%0A%0A%23%20%25%25manim%20-ql%20-t%20--progress_bar%20None%20--disable_caching%20-v%20ERROR%20RandomPointInSquare%0Aclass%20RandomPointInSquare%28Scene%29%3A%0A%20%20%20%20def%20construct%28self%29%3A%0A%20%20%20%20%20%20%20%20%23%20Set%20the%20background%20color%20to%20white%0A%20%20%20%20%20%20%20%20self.camera.background_color%20%3D%20WHITE%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%23%20Define%20the%20side%20length%20of%20the%20square%0A%20%20%20%20%20%20%20%20side_length%20%3D%206%0A%20%20%20%20%20%20%20%20radius%20%3D%20side_length%20/%202%0A%0A%20%20%20%20%20%20%20%20%23%20Create%20the%20square%0A%20%20%20%20%20%20%20%20square%20%3D%20Square%28side_length%3Dside_length,%20color%3DBLUE%29%0A%20%20%20%20%20%20%20%20square_label%20%3D%20MathTex%28r%22%5Cell%22,%20color%3DBLUE_D%29.next_to%28square,%20RIGHT%29%0A%0A%20%20%20%20%20%20%20%20%23%20Create%20the%20inscribed%20circle%0A%20%20%20%20%20%20%20%20circle%20%3D%20Circle%28radius%3Dradius,%20color%3DRED%29.move_to%28square.get_center%28%29%29%0A%0A%20%20%20%20%20%20%20%20%23%20Add%20the%20square%20and%20circle%20to%20the%20scene%0A%20%20%20%20%20%20%20%20self.play%28Create%28square%29%29%0A%20%20%20%20%20%20%20%20self.play%28Create%28circle%29%29%0A%20%20%20%20%20%20%20%20self.play%28Write%28square_label%29%29%0A%0A%20%20%20%20%20%20%20%20%23%20Simulate%20random%20points%0A%20%20%20%20%20%20%20%20num_points%20%3D%20100%0A%20%20%20%20%20%20%20%20points_inside%20%3D%200%0A%20%20%20%20%20%20%20%20for%20_%20in%20range%28num_points%29%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%23%20Generate%20a%20random%20point%0A%20%20%20%20%20%20%20%20%20%20%20%20x%20%3D%20np.random.uniform%28-radius,%20radius%29%0A%20%20%20%20%20%20%20%20%20%20%20%20y%20%3D%20np.random.uniform%28-radius,%20radius%29%0A%20%20%20%20%20%20%20%20%20%20%20%20point%20%3D%20Dot%28point%3D%5Bx,%20y,%200%5D%29%0A%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%23%20Check%20if%20the%20point%20is%20inside%20the%20circle%0A%20%20%20%20%20%20%20%20%20%20%20%20if%20x**2%20%2B%20y**2%20%3C%3D%20radius**2%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20point.set_color%28GREEN%29%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20points_inside%20%2B%3D%201%0A%20%20%20%20%20%20%20%20%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20point.set_color%28RED%29%0A%0A%20%20%20%20%20%20%20%20%20%20%20%20self.add%28point%29%0A%20%20%20%20%20%20%20%20%20%20%20%20self.wait%280.1%29&mode=edit&origin=opt-frontend.js&rawInputLstJSON=%5B%5D))

::::

By relating the chance of a random event to the quantity we want to compute, we can compute the quantity by simulating the random event. This method is called the Monte Carlo simulation:

::::{exercise}
:label: ex:ml1

Complete the following function to return an approximation of $\pi$ as follows:
1. Simulate the random process of picking a point from a square repeatedly `n` times by  
  generating the $x$ and $y$ coordinates uniformly randomly from a unit interval $[0,1)$.
2. Compute the fraction of times the point is in the first quadrant of the inscribed circle, as shown in the figure below.
3. Return $4$ times the fraction as the approximation.

:::{seealso} Buffon's needle

A similar way is to approximate $\pi$ is the [Buffon's needle](https://en.wikipedia.org/wiki/Buffon%27s_needle_problem). See [such a program](https://www.khanacademy.org/computer-programming/pi-by-buffons-needle/6695500989890560) written in javascript.

:::

::::

In [4]:
def approximate_pi(n):
    in_circle_count = 0
    for i in range(n):
            if random.random()**2 + random.random()**2 < 1:
                in_circle_count += 1
    result = 4*in_circle_count/n
    return result

print(f'Approximate: {approximate_pi(int(1e7))}\nGround truth: {math.pi}')

Approximate: 3.1415604
Ground truth: 3.141592653589793


In [5]:
%%timeit -n 1 -r 1
print(f"Approximate: {approximate_pi(10**7)}\nGround truth: {math.pi}")

Approximate: 3.1419936
Ground truth: 3.141592653589793
1.98 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


**How accurate is the approximation?**

The following uses a powerful library [NumPy](https://numpy.org/) to return a [$95\%$-confidence interval](http://onlinestatbook.com/2/estimation/mean.html#:~:text=To%20compute%20the%2095%25%20confidence,be%20between%20the%20cutoff%20points.).

In [4]:
def np_approximate_pi(n):
    in_circle = (np.random.random((n, 2)) ** 2).sum(axis=-1) < 1
    mean = 4 * in_circle.mean()
    std = 4 * in_circle.std() / n ** 0.5
    return np.array([mean - 2 * std, mean + 2 * std])

In [5]:
%%timeit -n 1 -r 1
interval = np_approximate_pi(10**7)
print(
    f"""95%-confidence interval: {interval}
Estimate: {interval.mean():.4f} ± {(interval[1]-interval[0])/2:.4f}
Ground truth: {math.pi}"""
)

95%-confidence interval: [3.14155943 3.14363577]
Estimate: 3.1426 ± 0.0010
Ground truth: 3.141592653589793
422 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


::::{note} Is Numpy `array` faster than Python `list`? 
:class: dropdown

The computation done using NumPy is over $5$ times faster despite the additional computation of the standard deviation.

::::

We can create $n$ further to obtain a more accurate solution:

In [6]:
interval = np_approximate_pi(10**8)
print(
    f"""95%-confidence interval: {interval}
Estimate: {interval.mean():.4f} ± {(interval[1]-interval[0])/2:.4f}
Ground truth: {math.pi}"""
)

95%-confidence interval: [3.14135122 3.14200806]
Estimate: 3.1417 ± 0.0003
Ground truth: 3.141592653589793


```{seealso}

There are faster methods to approximate $\pi$, such as the [BBP Formula](https://en.wikipedia.org/wiki/Bailey%E2%80%93Borwein%E2%80%93Plouffe_formula) and [Chudnovsky algorithm](https://en.wikipedia.org/wiki/Chudnovsky_algorithm), but the Monte-Carlo method is helpful in complicated situations.  
E.g., see the Monte Carlo simulation of a [real-life situation](https://www.youtube.com/watch?v=-fCVxTTAtFQ) in playing basketball:
> "When down by three and left with only 30 seconds, is it better to attempt a hard 3-point shot or an easy 2-point shot and get another possession?"   --LeBron James
```

In [7]:
%%html
<iframe width="912" height="513" src="https://www.youtube.com/embed/-fCVxTTAtFQ" allowfullscreen></iframe>

```{seealso}

How to use Monte Carlo Simulation to predict stock prices.
- You need to install library [yfinance](https://analyzingalpha.com/yfinance-python) if you want to try the code in the video below
```

In [8]:
%%html
<iframe width="912" height="513" src="https://www.youtube.com/embed/LWc-9v8RVwM" allowfullscreen></iframe>

## Linear Algebra

**How to solve a linear equation?**

Given the following linear equation in variable $x$ with real-valued coefficient $a$ and $b$,

$$ a x = b,$$
what is the value of $x$ that satisfies the equation?

::::{exercise}
:label: ex:ml2

Complete the following function to return either the unique solution of $x$ or `None` if a unique solution does not exist.

::::

In [9]:
def solve_linear_equation(a,b):
    if a !=0:
        return b/a
    else:
        return None
    
import ipywidgets as widgets
@widgets.interact(a=(0,5,1),b=(0,5,1))
def linear_equation_solver(a=2, b=3):
    print(f'''linear equation: {a}x = {b}
       solution: x = {solve_linear_equation(a,b)}''')

interactive(children=(IntSlider(value=2, description='a', max=5), IntSlider(value=3, description='b', max=5), …

**How to solve multiple linear equations?**

In the general case, we have a system of $m$ linear equations and $n$ variables:

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 + \dots + a_{0(n-1)} x_{n-1} &= b_0\\
a_{10} x_0 + a_{11} x_1 + \dots + a_{1(n-1)} x_{n-1} &= b_1\\
\vdots\kern2em &= \vdots\\
a_{(m-1)0} x_0 + a_{(m-1)1} x_1 + \dots + a_{(m-1)(n-1)} x_{n-1} &= b_{m-1}\\
\end{aligned}
$$
where
- $x_j$ for $j\in \{0,\dots,n-1\}$ are the variables, and
- $a_{ij}$ and $b_j$ for $i\in \{0,\dots,m-1\}$ and $j\in \{0,\dots,n-1\}$ are the coefficients.

A fundamental problem in linear algebra is to compute the unique solution to the system if it exists.

We will consider the simpler 2-by-2 system with 2 variables and 2 equations:

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 &= b_0\\
a_{10} x_0 + a_{11} x_1 &= b_1.
\end{aligned}
$$

To get an idea of the solution, suppose 

$$a_{00}=a_{11}=1, a_{01} = a_{10} = 0.$$
The system of equations become

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= b_0\\
\hphantom{x_0 +}  x_1 &= b_1,
\end{aligned}
$$
which gives the solution directly.

What about $a_{00}=a_{11}=2$ instead?

$$ \begin{aligned}
2x_0 \hphantom{+ x_1} &= b_0\\
\hphantom{2x_0 +}  2x_1 &= b_1,
\end{aligned}$$

To obtain the solution, we simply divide both equations by 2:

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= \frac{b_0}2\\
\hphantom{x_0 +}  x_1 &= \frac{b_1}2.
\end{aligned}
$$

What if $a_{01}=2$ instead?

$$ \begin{aligned}
2x_0 + 2x_1 &= b_0\\
\hphantom{2x_0 +}  2x_1 &= b_1\\
\end{aligned}
$$

The second equation gives the solution of $x_1$, and we can use the solution in the first equation to solve for $x_0$. More precisely:
- Subtract the second equation from the first one:

$$ \begin{aligned}
2x_0 \hphantom{+2x_1} &= b_0 - b_1\\
\hphantom{2x_0 +}  2x_1 &= b_1\\
\end{aligned}
$$
- Divide both equation by 2:

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= \frac{b_0 - b_1}2\\
\hphantom{x_0 +}  x_1 &= \frac{b_1}2\\
\end{aligned}
$$

::::{important} Row operations

A system of linear equations can be solved by the linear operations of 
1. multiplying an equation by a constant, and
2. subtracting one equation from another.
::::

How to write a program to solve a general 2-by-2 system? We will use the `numpy` library.

### Creating `numpy` arrays

**How to store the coefficients?**

In linear algebra, a system of equations such as

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 &= b_0\\
a_{10} x_0 + a_{11} x_1 &= b_1
\end{aligned}
$$
is written concisely in *matrix* form as $ \mathbf{A} \mathbf{x} = \mathbf{b} $:

$$\overbrace{\begin{bmatrix}
a_{00} & a_{01}\\
a_{10} & a_{11}
\end{bmatrix}}^{\mathbf{A}}
\overbrace{
\begin{bmatrix}
x_0\\
x_1
\end{bmatrix}}
^{\mathbf{x}}
= \overbrace{\begin{bmatrix}
b_0\\
b_1
\end{bmatrix}}^{\mathbf{b}},
$$
where
$ \mathbf{A} \mathbf{x}$ is the *matrix multiplication*

$$ \mathbf{A} \mathbf{x} = \begin{bmatrix}
a_{00} x_0 + a_{01} x_1\\
a_{10} x_0 + a_{11} x_1
\end{bmatrix}.
$$

We say that $\mathbf{A}$ is a [*matrix*](https://en.wikipedia.org/wiki/Matrix_(mathematics)) and its dimension/shape is $2$-by-$2$:
- The first dimension/axis has size $2$. We also say that the matrix has $2$ rows.
- The second dimension/axis has size $2$. We also say that the matrix has $2$ columns.
$\mathbf{x}$ and $\mathbf{b}$ are called column vectors, which are matrices with one column.

Consider the example
$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1,
\end{aligned}$$
or in matrix form with
$$ \begin{aligned}
\mathbf{A}&=\begin{bmatrix}
a_{00} & a_{01} \\
a_{10} & a_{11} 
\end{bmatrix} 
= \begin{bmatrix}
2 & 2 \\
0 & 2 
\end{bmatrix}\\
\mathbf{b}&=\begin{bmatrix}
b_0\\
b_1
\end{bmatrix} = \begin{bmatrix}
1\\
1
\end{bmatrix}\end{aligned}$$

Instead of using `list` to store the matrix, we will use a `numpy` array.

In [10]:
A = np.array([[2.,2],[0,2]])
b = np.array([1.,1])
A, b

(array([[2., 2.],
        [0., 2.]]),
 array([1., 1.]))

Compared to `list`, `numpy` array is often more efficient and has more useful attributes.

In [11]:
array_attributes = set(attr for attr in dir(np.array([])) if attr[0]!='_')
list_attributes = set(attr for attr in dir(list) if attr[0]!='_')
print('\nCommon attributes:\n',*(array_attributes & list_attributes))
print('\nArray-specific attributes:\n', *(array_attributes - list_attributes))
print('\nList-specific attributes:\n',*(list_attributes - array_attributes))


Common attributes:
 copy sort

Array-specific attributes:
 choose tostring put nonzero ptp tofile var flat flags mean argmin flatten strides partition argmax take diagonal T cumsum byteswap real sum min setflags dump item dot std size argpartition nbytes tolist ndim getfield dumps trace astype conjugate itemsize setfield view clip compress all reshape resize squeeze imag ravel round data any shape transpose max swapaxes dtype newbyteorder searchsorted prod conj argsort cumprod repeat ctypes fill base itemset tobytes

List-specific attributes:
 pop insert append extend remove reverse count index clear


The following attributes give the dimension/shape, number of dimensions, size, and datatype.

In [6]:
for array in A, b:
    print(f'''{array}
    shape: {array.shape}
    ndim: {array.ndim}
    size: {array.size}
    dtype: {array.dtype}
    ''')

[[2. 2.]
 [0. 2.]]
    shape: (2, 2)
    ndim: 2
    size: 4
    dtype: float64
    
[1. 1.]
    shape: (2,)
    ndim: 1
    size: 2
    dtype: float64
    


Note that the function `len` only returns the size of the first dimension:

In [9]:
assert A.shape[0] == len(A)
len(A)

2

Unlike `list`, every `numpy` array has a data type. For efficient computation/storage, numpy implements different data types with different storage sizes:
* integer: `int8`, `int16`, `int32`, `int64`, `uint8`, ...
* float: `float16`, `float32`, `float64`, ...
* complex: `complex64`, `complex128`, ...
* boolean: `bool8`
* Unicode: `string`
* Object: `object`

E.g., `int64` is the 64-bit integer. Unlike `int`, `int64` has a range.

In [12]:
np.int64?
print(f'range: {np.int64(-2**63)} to {np.int64(2**63-1)}')
np.int64(2**63)   # overflow error

range: -9223372036854775808 to 9223372036854775807


OverflowError: Python int too large to convert to C long

We can use the `astype` method to convert the data type:

In [13]:
A_int64 = A.astype(int)       # converts to int64 by default
A_float32 = A.astype(np.float32)  # converts to float32
for array in A_int64, A_float32:
    print(array, array.dtype)

[[2 2]
 [0 2]] int64
[[2. 2.]
 [0. 2.]] float32


We have to be careful about assigning items of different types to an array.

In [14]:
A_int64[0,0] = 1
print(A_int64)
A_int64[0,0] = 0.5
print(A_int64)                # intended assignment fails, can't assign floating point number to A whose data type is int64
np.array([int(1), float(1)])  # will be all floating point numbers

[[1 2]
 [0 2]]
[[0 2]
 [0 2]]


array([1., 1.])

::::{exercise}
:label: ex:ml3

Create a heterogeneous NumPy array to store both integers and strings:
```python
[0, 1, 2, 'a', 'b', 'c']
```
:::{hint}
:class: dropdown

Use the [`object` type](https://numpy.org/devdocs/reference/arrays.scalars.html#numpy.object_).
:::

::::

In [15]:
#np.object?

heterogeneous_np_array = np.array([*range(3),*'abc'],dtype=object)

heterogeneous_np_array

array([0, 1, 2, 'a', 'b', 'c'], dtype=object)

`numpy` provides many functions to create an array:

In [17]:
#np.zeros?
np.zeros(0), np.zeros(1), np.zeros((2,3,4))  # Dimension can be higher than 2

(array([], dtype=float64),
 array([0.]),
 array([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],
 
        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]]))

In [18]:
#np.ones?
np.ones(0, dtype=int), np.ones((2,3,4), dtype=int)  # initialize values to int 1

(array([], dtype=int64),
 array([[[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]],
 
        [[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]]]))

In [20]:
#np.eye?
np.eye(0), np.eye(1), np.eye(2), np.eye(3)  # identity matrices

(array([], shape=(0, 0), dtype=float64),
 array([[1.]]),
 array([[1., 0.],
        [0., 1.]]),
 array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]))

In [21]:
#np.diag?
np.diag(range(1)), np.diag(range(2)), np.diag(range(4))  # diagonal matrices

(array([[0]]),
 array([[0, 0],
        [0, 1]]),
 array([[0, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 2, 0],
        [0, 0, 0, 3]]))

In [26]:
#np.empty?
np.empty(0), np.empty((2,3,4), dtype=int)  # create array faster without initialization

(array([], dtype=float64),
 array([[[                  0,                   0,                   0,
                            0],
         [4607182418800017408, 4607182418800017408, 4607182418800017408,
          4607182418800017408],
         [4611686018427387904, 4611686018427387904, 4611686018427387904,
          4611686018427387904]],
 
        [[                  0, 4607182418800017408, 4611686018427387904,
          4613937818241073152],
         [                  0, 4607182418800017408, 4611686018427387904,
          4613937818241073152],
         [                  0, 4607182418800017408, 4611686018427387904,
          4613937818241073152]]]))

`numpy` also provides functions to build an array using rules.

In [16]:
#np.arange?
np.arange(5), np.arange(4,5), np.arange(4.5,5.5,0.5)  # like range but allow non-integer parameters

(array([0, 1, 2, 3, 4]), array([4]), array([4.5, 5. ]))

In [9]:
print(*range(5))
print(*range(4,5))
print(*range(4,10,2))

0 1 2 3 4
4
4 6 8


In [10]:
#np.linspace?
np.linspace(4,5), np.linspace(4,5,6), np.linspace(4,5,11)  # can specify number of points instead of step, default 50

(array([4.        , 4.02040816, 4.04081633, 4.06122449, 4.08163265,
        4.10204082, 4.12244898, 4.14285714, 4.16326531, 4.18367347,
        4.20408163, 4.2244898 , 4.24489796, 4.26530612, 4.28571429,
        4.30612245, 4.32653061, 4.34693878, 4.36734694, 4.3877551 ,
        4.40816327, 4.42857143, 4.44897959, 4.46938776, 4.48979592,
        4.51020408, 4.53061224, 4.55102041, 4.57142857, 4.59183673,
        4.6122449 , 4.63265306, 4.65306122, 4.67346939, 4.69387755,
        4.71428571, 4.73469388, 4.75510204, 4.7755102 , 4.79591837,
        4.81632653, 4.83673469, 4.85714286, 4.87755102, 4.89795918,
        4.91836735, 4.93877551, 4.95918367, 4.97959184, 5.        ]),
 array([4. , 4.2, 4.4, 4.6, 4.8, 5. ]),
 array([4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. ]))

In [29]:
#np.fromfunction?
np.fromfunction(lambda i, j: i * j, (3,4))  # can initialize using a function

array([[0., 0., 0., 0.],
       [0., 1., 2., 3.],
       [0., 2., 4., 6.]])

We can also reshape an array using the `reshape` method/function:

In [17]:
array = np.arange(2*3)
#array.reshape?
(array, 
 array.reshape(3,2),array.reshape(2,3))

(array([0, 1, 2, 3, 4, 5]),
 array([[0, 1],
        [2, 3],
        [4, 5]]),
 array([[0, 1, 2],
        [3, 4, 5]]))

In [18]:
%%optlite -l -h 500
import numpy as np
a = np.arange(2 * 3 * 4)
# default C ordering
c = a.reshape((2, 3, -1), order="C")
# F ordering
f = a.reshape((2, 3, -1), order="F")

OPTWidget(value=None, height=500, script='import numpy as np\na = np.arange(2 * 3 * 4)\n# default C ordering\n…

The C ordering and F ordering can be implemented using list comprehension:

In [12]:
%%optlite -l -h 500
import numpy as np
a = np.arange(2 * 3 * 4)
# last axis index changes fastest
c = np.array(
    [[[a[i*3*4+j*4+k] 
       for k in range(4)]
      for j in range(3)]
     for i in range(2)])
# first axis index changes fastest
f = np.array(
    [[[a[i+j*2+k*3*2]
       for k in range(4)]
      for j in range(3)]
     for i in range(2)])

OPTWidget(value=None, height=500, script='import numpy as np\na = np.arange(2 * 3 * 4)\n# last axis index chan…

::::{note} How to distinguish between C and F orderings easily?

When restricted to 2-dimension:

- C-style (row-major) order is like English writing, where one writes row by row.
- F-style (column-major) order is like traditional Chinese writing, where one writes column by column.

::::

`ravel` is a particular case of reshaping an array to one dimension.  
A similar function `flatten` returns a copy of the array but `ravel` and `reshape` returns a dynamic view whenever possible.

In [19]:
array = np.arange(2 * 3 * 4).reshape(2, 3, 4)
array, array.ravel(), array.reshape(-1), array.ravel(order="F")

(array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23]),
 array([ 0, 12,  4, 16,  8, 20,  1, 13,  5, 17,  9, 21,  2, 14,  6, 18, 10,
        22,  3, 15,  7, 19, 11, 23]))

::::{exercise}
:label: ex:ml4

Correct the following function to print every element of an array line-by-line.
```python
def print_array_entries_line_by_line(array):
    for i in array:
        print(i)
```

::::

In [20]:
def print_array_entries_line_by_line(array):

    for i in array.flatten():
        print(i)
    
print_array_entries_line_by_line(np.arange(2*3).reshape(2,3))

0
1
2
3
4
5


### Operating on `numpy` arrays

**How to verify the solution of a system of linear equations?**

Before solving the system of linear equations, let us try to verify a solution to the equations:

$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1
\end{aligned}
$$

`numpy` provides the function `matmul` and the operator `@` for matrix multiplication.

In [21]:
A = np.array([[2.,2],[0,2]])
b = np.array([1.,1])
A, b
print(np.matmul(A,np.array([0,0])) == b)
print(A @ np.array([0,0.5]) == b)

[False False]
[ True  True]


Note that the comparison on `numpy` arrays returns a boolean array instead of a boolean value, unlike the comparison operations on lists.

To check whether all items are true, we use the `all` method.

In [34]:
print((np.matmul(A,np.array([0,0])) == b).all())
print((A @ np.array([0,0.5]) == b).all())

False
True


**How to concatenate arrays?**

We will operate on an augmented matrix of the coefficients:

$$ \begin{aligned} \mathbf{C} &= \begin{bmatrix}
\mathbf{A} & \mathbf{b}
\end{bmatrix}\\
&= \begin{bmatrix}
a_{00} & a_{01} & b_0 \\
a_{10} & a_{11} & b_1
\end{bmatrix} 
\end{aligned}
$$


`numpy` provides functions to create block matrices:

In [28]:
#np.block?
C = np.block([A,b.reshape(-1,1)])  # reshape to ensure same ndim
print(A)
print(b)
print(C)

[[2. 2.]
 [0. 2.]]
[1. 1.]
[[2. 2. 1.]
 [0. 2. 1.]]


To stack an array along different axes:

In [29]:
array = np.arange(2*3).reshape(2,3)
for concat_array in [array,
                     np.hstack((array,array)),   # stack along the first axis
                     np.vstack((array,array)),                  # second axis
                     np.concatenate((array,array), axis=-1),    # last axis
                     np.stack((array,array), axis=0)]:          # new axis
    print(concat_array, '\nshape:', concat_array.shape)

[[0 1 2]
 [3 4 5]] 
shape: (2, 3)
[[0 1 2 0 1 2]
 [3 4 5 3 4 5]] 
shape: (2, 6)
[[0 1 2]
 [3 4 5]
 [0 1 2]
 [3 4 5]] 
shape: (4, 3)
[[0 1 2 0 1 2]
 [3 4 5 3 4 5]] 
shape: (2, 6)
[[[0 1 2]
  [3 4 5]]

 [[0 1 2]
  [3 4 5]]] 
shape: (2, 2, 3)


**How to perform arithmetic operations on a `numpy` array?**

To divide all the coefficients by $2$, we can simply write:

In [30]:
D = C / 2
print(C)
print(D)

[[2. 2. 1.]
 [0. 2. 1.]]
[[1.  1.  0.5]
 [0.  1.  0.5]]


Arithmetic operations on `numpy` arrays apply if the arrays have compatible dimensions. Two dimensions are compatible when
- they are equal, except for
- components equal to 1.

`numpy` uses [broadcasting rules](https://numpy.org/doc/stable/user/basics.broadcasting.html#general-broadcasting-rules) to stretch the axis of size 1 up to match the corresponding axis in other arrays.  
`C / 2` is a example where the second operand $2$ is broadcasted to a $2$-by-$2$ matrix before the elementwise division. Another example is as follows. 

In [31]:
three_by_one = np.arange(3).reshape(3,1)
one_by_four = np.arange(4).reshape(1,4)
print(f'''
{three_by_one}
*
{one_by_four}
==
{three_by_one * one_by_four}
''')


[[0]
 [1]
 [2]]
*
[[0 1 2 3]]
==
[[0 0 0 0]
 [0 1 2 3]
 [0 2 4 6]]



Next, to subtract the second row of the coefficients from the first row:

In [32]:
D[0,:] = D[0,:] - D[1,:]
D

array([[1. , 0. , 0. ],
       [0. , 1. , 0.5]])

Using this indexing technique, it is easy extract the last column as the solution to the system of linear equations:

In [33]:
print(D)
x = D[:,-1]
x

[[1.  0.  0. ]
 [0.  1.  0.5]]


array([0. , 0.5])

This gives the desired solution $x_0=0$ and $x_1=0.5$ for

$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1\\
\end{aligned}$$

`numpy` indexing and slicing

In [41]:
#one-dimension array
a = np.arange(10)  
a_slice = a[2:7:2]   # start from index 2 and stop to index 7 (exclusive) with a step of 2
print(a)
print(a_slice)

[0 1 2 3 4 5 6 7 8 9]
[2 4 6]


In [42]:
#two-dimension array
a = np.array([[1,2,3],[3,4,5],[4,5,6]])
print(a)
print('slice from a[1:]:')
print(a[1:])

[[1 2 3]
 [3 4 5]
 [4 5 6]]
slice from a[1:]:
[[3 4 5]
 [4 5 6]]


... expands to selecting all elements of all previous dimensions

In [34]:
a = np.array([[1,2,3],[3,4,5],[4,5,6]])  
print("a is: \n",a)
print("select the second column:\n",a[...,1])   # select the second column
print("select the second row:\n", a[1,...])   # select the second row
print("select from the second column to the last column:\n", a[...,1:])  # select from the second column to the last column

a is: 
 [[1 2 3]
 [3 4 5]
 [4 5 6]]
select the second column:
 [2 4 5]
select the second row:
 [3 4 5]
select from the second column to the last column:
 [[2 3]
 [4 5]
 [5 6]]


In [35]:
a = np.array([[0,1,2],[3,4,5],[6,7,8],[9,10,11]]) 
print(a)
print('all the elements >5:')
print(a[a > 5])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
all the elements >5:
[ 6  7  8  9 10 11]


Finally, the following function solves a system of 2 linear equations with 2 variables.

In [36]:
def solve_2_by_2_system(A,b):
    '''Returns the unique solution of the linear system, if exists, 
    else returns None.'''
    C = np.hstack((A,b.reshape(-1,1)))
    if C[0,0] == 0: C = C[(1,0),:]
    if C[0,0] == 0: return None
    C[0,:] = C[0,:] / C[0,0]
    C[1,:] = C[1,:] - C[0,:] * C[1,0]
    if C[1,1] == 0: return None
    C[1,:] = C[1,:] / C[1,1]
    C[0,:] = C[0,:] - C[1,:] * C[0,1]
    return C[:,-1]

In [37]:
# tests
for A in (np.eye(2),
          np.ones((2,2)),
          np.stack((np.ones(2),np.zeros(2))),
          np.stack((np.ones(2),np.zeros(2)),axis=1)):
    print(f'A={A}\nb={b}\nx={solve_2_by_2_system(A,b)}\n')

A=[[1. 0.]
 [0. 1.]]
b=[1. 1.]
x=[1. 1.]

A=[[1. 1.]
 [1. 1.]]
b=[1. 1.]
x=None

A=[[1. 1.]
 [0. 0.]]
b=[1. 1.]
x=None

A=[[1. 0.]
 [1. 0.]]
b=[1. 1.]
x=None



### Universal functions

A universal function (or *ufunc* for short) is a function that operates on ndarrays in an element-by-element fashion
* NumPy provides familiar mathematical functions, such as
   * *np.exp* : $e^x$, where $e=2.718281$ (also called Euler's number)
   * *np.sqrt*
   * *np.sin*
   * *np.cos*
* Within NumPy, these functions operate elementwise on an array, producing an array as output.

In [38]:
import numpy as np
a = np.array([[1, 2, 3],[4,5,6]])
print('a:')
print(a)
print('exp(a):')
print(np.exp(a))
print('square root of a:')
print(np.sqrt(a))
print('sin(a):')
print(np.sin(a))
print('cos(a):')
print(np.cos(a))

a:
[[1 2 3]
 [4 5 6]]
exp(a):
[[  2.71828183   7.3890561   20.08553692]
 [ 54.59815003 148.4131591  403.42879349]]
square root of a:
[[1.         1.41421356 1.73205081]
 [2.         2.23606798 2.44948974]]
sin(a):
[[ 0.84147098  0.90929743  0.14112001]
 [-0.7568025  -0.95892427 -0.2794155 ]]
cos(a):
[[ 0.54030231 -0.41614684 -0.9899925 ]
 [-0.65364362  0.28366219  0.96017029]]


[fromfunction()](https://numpy.org/doc/stable/reference/generated/numpy.fromfunction.html)
- Construct an array by executing a function over each coordinate.

In [13]:
print(np.fromfunction(lambda i, j: i == j, (3, 3), dtype=int))
print(np.fromfunction(lambda i, j: i + j, (3, 3), dtype=int))

[[ True False False]
 [False  True False]
 [False False  True]]
[[0 1 2]
 [1 2 3]
 [2 3 4]]


Why does the first line of code below return two arrays but the second code return only one array? Shouldn't the first line of code return the following?
```Python
array([[(0,1), (0,2), (0,3)],
       [(1,1), (1,2), (1,3)]])
```

In [14]:
print(np.fromfunction(lambda i,j:(i,j), (2,3), dtype=int))
print(np.fromfunction(lambda i,j:(i*j), (2,3), dtype=int))

(array([[0, 0, 0],
       [1, 1, 1]]), array([[0, 1, 2],
       [0, 1, 2]]))
[[0 0 0]
 [0 1 2]]


In [15]:
#lambda i,j:(i,j) is equivalent to the following function
def f1(i,j):
    return (i,j)  #it returns a tuple


#because the shape is (2,3), so i and j are:
i=np.array([[0, 0 ,0],
   [1, 1, 1]])
j=np.array([[0, 1, 2],
   [0, 1, 2]])

#the function will use i and j (both are arrays) as argument
result1 = f1(i,j)
print(result1)

(array([[0, 0, 0],
       [1, 1, 1]]), array([[0, 1, 2],
       [0, 1, 2]]))


From the documentation, `fromfunction` applies the given function to the two arrays as arguments.
- The first line of code returns a tuple of the arrays.
- The second line of code multiplies the two arrays to give one array, according to how multiplication works for numpy arrays.

Indeed, `numpy` implements [universal/vectorized functions/operators](https://numpy.org/doc/stable/reference/ufuncs.html) that take arrays as arguments and perform operations with appropriate broadcasting rules. The following is an example that uses the universal function `np.sin`:

In [42]:
import matplotlib.pyplot as plt

@widgets.interact(a=(0, 5, 1), b=(-1, 1, 0.1))
def plot_sin(a=1, b=0):
    x = np.linspace(0, 2 * math.pi)
    plt.plot(x, np.sin(a * x + b * math.pi))  # np.sin, *, + are universal functions
    plt.title(r"$\sin(ax+b\pi)$")
    plt.xlabel(r"$x$ (radian)")

interactive(children=(IntSlider(value=1, description='a', max=5), FloatSlider(value=0.0, description='b', max=…

In addition to making the code shorter, universal functions are both efficient and flexible. (Recall the Monte Carlo simulation to approximate $\pi$.)

::::{exercise}
:label: ex:ml5

Explain the universal functions used in the following Monte Carlo simulation:

```python
def np_approximate_pi(n):
    in_circle = (np.random.random((n,2))**2).sum(axis=-1) < 1
    mean = 4 * in_circle.mean()
    std = 4 * in_circle.std() / n**0.5
    return np.array([mean - 2*std, mean + 2*std])
```

::::

- `random.random` generates a numpy array for $n$ points in the unit square randomly.
- `sum` sums up the element along the last axis to give the squared distance.
- `<` returns the boolean array indicating whether each point is in the first quadrant of the inscribed circle.
- `mean` and `std` returns the mean and standard deviation of the boolean array with True and False interpreted as 1 and 0 respectively.